In [1]:
library(tidyverse)
library(magrittr)
library(pomp)
library(foreach)
library(doSNOW)
library(magrittr)
library(lubridate)
rm(list = ls())
Sys.setlocale("LC_ALL","C")

departement <- 'Artibonite'

# function to convert dates to fractions of years for model
dateToYears <- function(date, origin = as.Date("2014-01-01"), yr_offset = 2014) {
  julian(date, origin = origin)/365.25 + yr_offset
}

yearsToDate <- function(year_frac, origin = as.Date("2014-01-01"), yr_offset = 2014.0) {
  as.Date((year_frac - yr_offset) * 365.25, origin = origin)
}

yearsToDateTime <- function(year_frac, origin = as.Date("2014-01-01"), yr_offset = 2014.0) {
  as.POSIXct((year_frac - yr_offset) * 365.25 * 3600 * 24, origin = origin)
}

# Load data ---------------------------------------------------------------

# cholera case data from the 2014-2015 epidemic in Juba (South Soudan)
#cases <- read_csv("haiti-data/fromAzman/cases.csv") %>%  
#select('date', departement) %>% 
#     mutate(date = as.Date(date, format = "%Y-%m-%d"),
#            time = dateToYears(date))
# Javier says the second is better
cases <- read_csv("haiti-data/fromAzman/cases.csv")  %>% 
gather(dep, cases, -date) %>% 
group_by(dep) %>% 
filter(dep == "Artibonite") %>% 
     mutate(date = as.Date(date, format = "%Y-%m-%d"),
            time = dateToYears(date))

# get the time of the first datapoint (use %>% filter(time > 2015) to constraint)
t_first_datapnt <- cases %>% slice(1) %>% .[["time"]]

# Estimates of daily rainfall 
#rain <- read_csv("haiti-data/fromAzman/rainfall.csv") %>%  
#select('date', departement) %>%
#mutate(date = as.Date(date, format = "%Y-%m-%d"),
#   time = dateToYears(date)) 

# value of maximal event OK ^
#max_rain <- rain %>%
#filter(year(date)==2015 & month(date) < 10) %>% 
#select(departement) %>% 
#max()


# standardize rainfall  TODO 
#rain %<>% mutate(rain_std = rain/max_rain)


rain <- read_csv("haiti-data/fromAzman/rainfall.csv")  %>% 
gather(dep, rain, -date) %>% 
group_by(dep) %>% 
mutate(max_rain = max(rain), rain_std = rain/max_rain) %>%
filter(dep == "Artibonite") %>% 
     mutate(date = as.Date(date, format = "%Y-%m-%d"),
            time = dateToYears(date))


make_plots <- F
if(make_plots) {
  # plot the data
  p.data <- ggplot(cases, aes(x = time, y = cases)) + 
  geom_line() + 
  geom_bar(data = rain, aes(y = rain_std  * 100), stat = "identity", fill = "blue") +
  scale_x_continuous(breaks = c(2014, 2015)) 
  
  p.data
}


# Model specification -----------------------------------------------------

## state variable names:
# S:   Susceptibles
# I:   Exposed Infected
# A:   Exposed Asymptomatics (infected but still asymptomatic)
# RI1: Recovered from exposed infected
# RI2: Recovered from exposed infected
# RI3: Recovered from exposed infected
# RA1: Recovered from exposed asymptomatics
# RA2: Recovered from exposed asymptomatics
# RA3: Recovered from exposed asymptomatics
# B:   Bacterial concentration in the environment
# C:   Cumulative cases

## data names:
# cases: reported suspected cholera cases (weekly)

## covariate names:
# rain: estimated weekly precipitation standardized with the maximal observation

## parameter names for a departement:
### Pop dynamics
# H:        total population
# D:        density
# gammaI:   recovery from infection
# gammaA:   recovery from asymptomatcs
# mu:       natural mortylity
# alpha:    cholera-induced mortality
# rhoI:      loss of aquired immunity for infected
# rhoA:      loss of aquired immunity for asymptomatics
### Infection dynamics
#### Basic SIRB
# sigma:    sympotmatic to asymptomatic ratio
# betaB:   envionmental (indirect) transmission coefficient
### Bacterial pop dynamics
# mu_B:     bacterial mortality
# thetaA:    bacterial output per asymtomatic person
# thetaI:    bacterial output per infected person
#### Rainfall effects
# lambda: multiplicative rainfall effect on contamination
# r:  power of rainfall effect on contamination
#### Extra-demographic stochasticity
# std_W:    standard deviation of the weiner process to perturb the foi       
### Vaccination
# r_v:      rate of vaccination
# eff_v:    vaccine efficacy
# rho_v:    loss of vaccine-induced immunity
### Measurement model
# epsilon:  under-rerpoting fraction

# Set variables -----------------------------------------------------------

# define stat variable names OK
state_names <- c("S", "I", "A", "RI1", "RI2", "RI3", "RA1", "RA2", "RA3", "B", "C", "W")

# define parameter names for pomp
## process model parameters names to estimate OK
param_proc_est_names <- c("sigma", "betaB", "r", "mu_B", "thetaA", "thetaI", "lambda", "rhoA", "rhoI", "std_W", "epsilon","k")

## initial value parameters to estimate OK
param_iv_est_names <- c("RI1_0")

## fixed process model parameters  OK
param_proc_fixed_names <- c("H", "D", "mu", "alpha", "gammaI", "gammaA")

## fixed initial value parameters OK
param_iv_fixed_names <- c("I_0","A_0", "B_0", "RI1_0", "RI2_0", "RI3_0", 
    "RA1_0", "RA2_0", "RA3_0")

# all paramter names to estimate OK
param_est_names <- c(param_proc_est_names, param_iv_est_names)
# all fixed parameters OK
param_fixed_names <- c(param_proc_fixed_names, param_iv_fixed_names)

# all param names OK
param_names <- c(param_est_names, param_fixed_names)

# names of parameters that are rates MAYBE
param_rates_in_days_names <- c("mu", "alpha", "gammaI", "gammaA", "rhoI", "rhoA")

# names of rate parameters WUT
param_rate_names <- param_names[!str_detect(param_names, "_0|H|k|epsilon|eff|t_|std_W|sigma|alpha_|lambda")] #???


# declare matrix in C for the recoveries in 2014
#cases_2014 <- cases %>% filter(year(date) == 2014)
#cases2014.string <- foreach(r = iter(cases_2014, by = "row"),
#    .combine = c) %do% {
#  sprintf(" {%f, %f} ", r$time, r$cases)
#  } %>% 
#  str_c(collapse = ", \n")

#  matrix_cases2014.string <- str_c(sprintf("double cases2014[%i][%i] = {\n", nrow(cases_2014), 2),
#   cases2014.string,
#   " \n };")


# Measurment model  -------------------------------------------------------

# measurement model
## density
 
## NegBinomial density (if k -> inf then becomes Poisson) OK
dmeas <- Csnippet("
  double mean_cases = epsilon * C;
  if (ISNA(cases)) {
    lik = (give_log) ? 0 : 1;
    } else {
        if (S < 10000) {
          lik = (give_log) ? -99999 : 1.0e-18;
          } else {
              lik = dnbinom_mu(cases, k, mean_cases, give_log) ;
          }
      }
      ")

## NegBinomial simulator OK
rmeas <- Csnippet("
  double mean_cases = epsilon * C;
  cases = rnbinom_mu(k, mean_cases);
  ")

# Process model ----------------------------------------------------------------- OK

sirb.rproc <- Csnippet("
  double foi, foi_stoc; // force of infection and its stochastic version
  double dw;            // extra-demographic stochasticity on foi
  double dB;            // deterministic forward time difference of bacteria in the environment
  double k1, k2, k3, k4;  // coefficients of  the Runge-Kutta method
  double r_v_wdn;       // rate of vaccination: 0 if out of time window, r_v if not
  double rate[19];      // vector of all rates in model
  double dN[19];        // vector of transitions between classes during integration timestep

  // force of infection
  foi = betaB * (B / (1 + B));

  if(std_W > 0.0) {
    // white noise (extra-demographic stochasticity)
    dw = rgammawn(std_W, dt);
    // apply stochasticity
    foi_stoc = foi * dw/dt;
    } else {
        foi_stoc = foi;
    }

    // vaccination window
    if (t >= t_vacc_start && t <= (t_vacc_end + dt)) 
    r_v_wdn = (r_v / (S + E + R));
    else 
    r_v_wdn = 0.0;

    // define transition rates for each type of event
    // S compartment
    rate[0] = sigma * foi_stoc;   // infections
    rate[1] = (1 - sigma) * foi_stoc;   // asymptomatic infections
    // I compartment
    rate[2] = mu;         // natural deaths
    rate[3] = alpha;      // cholera-induced deaths
    rate[4] = gammaI;      // recovery from infection
    // A compartment (not in order because was added after initial model formulation)
    rate[5] = mu;        // natural death
    rate[6] = gammaA;       // symptoms development
    // RI1,2,3 compartment
    rate[7] = 3*rhoI;        // loss of natural immunity
    rate[8] = mu;         // natural death
    // RI2 compartment
    rate[9] = 3*rhoI;        // loss of natural immunity
    rate[10] = mu;
    // RI3 compartment
    rate[11] = 3*rhoI;        // loss of natural immunity
    rate[12] = mu;
    // RA1,2,3 compartment
    rate[13] = 3*rhoA;        // loss of natural immunity
    rate[14] = mu;         // natural death
    // RA2 compartment
    rate[15] = 3*rhoA;        // loss of natural immunity
    rate[16] = mu;
    // RA3 compartment
    rate[17] = 3*rhoA;        // loss of natural immunity
    rate[18] = mu;


    // simulate all transitions
    reulermultinom(2, S, &rate[0], dt, &dN[0]);
    reulermultinom(3, I, &rate[2], dt, &dN[2]);
    reulermultinom(2, A, &rate[5], dt, &dN[5]);
    reulermultinom(2, RI1, &rate[7], dt, &dN[7]);
    reulermultinom(2, RI2, &rate[9], dt, &dN[9]);
    reulermultinom(2, RI3, &rate[11], dt, &dN[11]);
    reulermultinom(2, RA1, &rate[13], dt, &dN[13]);
    reulermultinom(2, RA2, &rate[15], dt, &dN[15]);
    reulermultinom(2, RA3, &rate[17], dt, &dN[17]);


    // bacteria as continous state variable
    // implement Runge-Kutta integration assuming S, I, R, V* stay constant during dt
    k1 = dt * fB(I, A, B, mu_B, thetaI, thetaA, lambda, rain, r, density);
    k2 = dt * fB(I, A, B, mu_B, thetaI, thetaA, lambda, rain, r, density);
    k3 = dt * fB(I, A, B, mu_B, thetaI, thetaA, lambda, rain, r, density);
    k4 = dt * fB(I, A, B, mu_B, thetaI, thetaA, lambda, rain, r, density);
    // bacteria increment
    dB = (k1 + 2*k2 + 2*k3 + k4) / 6.0;

    // update state variables

    I   += dN[0] - dN[2] - dN[3] - dN[4];
    A   += dN[1] - dN[5] - dN[6];
    RI1 += dN[4] - dN[7] - dN[8];
    RI2 += dN[7] - dN[9] - dN[10];
    RI3 += dN[9] - dN[11] - dN[12];
    RA1 += dN[6] - dN[13] - dN[14];
    RA2 += dN[13] - dN[15] - dN[16];
    RA3 += dN[15] - dN[17] - dN[18];
    C   +=  dN[0];
    W   +=  (dw - dt)/std_W;  // standardized i.i.d. white noise
    B += (((dB) < -B) ? (-B + 1.0e-3) : (dB)); // condition to ensure B>0

    // susceptibles so as to match total population
    S = nearbyint(H - I - A - R1I - R2I - R3I - R1A - R2A - R3A);
    ")

# C function to compute the time-derivative of bacterial concentration OK
derivativeBacteria.c <- " double fB(int I, int A, double B, 
    double mu_B, double thetaI, double thetaA, double lambda, double rain, double r, double density) {
  double dB;
  dB = -mu_B * B +  (1 + lambda * pow(rain, r)) * density * (thetaI * (double) I + thetaA * (double) A);
  return(dB);
};
"

# C function to compute the initial number of recovered at the start of the simulations given sigma and epsilon.
# computeRecovered2014.c <- "int computeRecovered(double t0, double R_0_2014,  int n_cases2014, double cases2014[][2], double sigma, double rho, double epsilon){
#   double R_0_2015 = 0;
#   // loop over reported cases in 2014 and compute remaning in 205
#   for(int i = 0; i < n_cases2014; i++){
#     R_0_2015 += cases2014[i][1] * (1-sigma)/sigma/epsilon  * exp((cases2014[i][0] - t0) * rho);
# }
# // add the calibrated IC for R in the beginning of 2014 
# R_0_2015 += R_0_2014 * exp((cases2014[0][0] - t0) * rho);

# return(nearbyint(R_0_2015));
# };
# "
# Deterministic skeleton -------------------------------------------------- OK
sirb.skeleton <- Csnippet("
 double foi; // force of infection and its stochastic version
 double r_v_wdn;       // rate of vaccination: 0 if out of time window, r_v if not
 double rate[19];      // vector of all rates in model

 // force of infection
 foi = betaB * (B / (1 + B));

 // vaccination window
 if (t >= t_vacc_start && t <= (t_vacc_end + 1/365.25)) 
 r_v_wdn = (r_v / (S + E + R));
 else 
 r_v_wdn = 0;

 // define transition rates for each type of event
 // S compartment
 rate[0] = sigma * foi_stoc;   // infections
 rate[1] = (1 - sigma) * foi_stoc;   // asymptomatic infections
 // I compartment
 rate[2] = mu;         // natural deaths
 rate[3] = alpha;      // cholera-induced deaths
 rate[4] = gammaI;      // recovery from infection
 // A compartment (not in order because was added after initial model formulation)
 rate[5] = mu;        // natural death
 rate[6] = gammaA;       // symptoms development
 // RI1,2,3 compartment
 rate[7] = 3*rhoI;        // loss of natural immunity
 rate[8] = mu;         // natural death
 // RI2 compartment
 rate[9] = 3*rhoI;        // loss of natural immunity
 rate[10] = mu;
 // RI3 compartment
 rate[11] = 3*rhoI;        // loss of natural immunity
 rate[12] = mu;
 // RA1,2,3 compartment
 rate[13] = 3*rhoA;        // loss of natural immunity
 rate[14] = mu;         // natural death
 // RA2 compartment
 rate[15] = 3*rhoA;        // loss of natural immunity
 rate[16] = mu;
 // RA3 compartment
 rate[17] = 3*rhoA;        // loss of natural immunity
 rate[18] = mu;

 // update state variables
 DI  = rate[0] * S - (rate[2] + rate[3] + rate[4]) * I;
 DA  = rate[1] * S - (rate[5] + rate[6]) * A;
 DRI1  = rate[4] * I - (rate[7] + rate[8]) * RI1;
 DRI2  = rate[7] * RI1 - (rate[9] + rate[10]) * RI2;
 DRI3  = rate[9] * RI2 - (rate[11] + rate[12]) * RI2;
 DRA1  = rate[6] * A - (rate[13] + rate[14]) * RA1;
 DRA2  = rate[13] * I - (rate[15] + rate[16]) * RI1;
 DRA3  = rate[15] * I - (rate[17] + rate[18]) * RI1;

 DC  = rate[0] * S;
 DW  = foi;  // standardized i.i.d. white noise
 DVtot = rate[2] * S + rate[18] * E + rate[8] * R;

 // bacteria as continous state variable
 DB = -mu_B * B + theta * (1 + lambda_R * pow(rain, r)) * (thetaA * A + thetaI * I);
 // susceptibles so as to match total population
 DS = -(DA + DI + DRI1 + DRI2 +DRI3 +DRA1 + DRA2 + DRA3);
 ")





# Initializer -------------------------------------------------------------

# initalizeStates <- Csnippet("
#   double m = H ;// /(S_0+E_0+I_0+R_0);
#   A   = nearbyint(A_0 * m/epsilon);
#   I   = nearbyint(I_0 * m/epsilon);
#   R   = computeRecovered(t0, R_0 * H, n_cases2014, cases2014, sigma, rho, epsilon);
#   R   = ((R >= H) ? (m - E - I - 100.0) : (R)); // remove 100 so that S > 0 if the predicted R > H
#   S   = nearbyint(m - E - I - R);
#   B   = 2.0/epsilon * theta/mu_B; // custom initial conditions equivalent to the 'forcing' in the continous model
#   C   = 0;
#   W   = 0;
#   Vtot  = 0;
#   ")

# TODO
initalizeStates <- Csnippet("
  double m = H ;// /(S_0+E_0+I_0+R_0);
  A   = 0;
  I   = 0;
  RI1   = 100;
  RI2   = 100;
  RI3   = 100;
  RA1   = 100;
  RA2   = 100;
  RA3   = 100;
  S   = nearbyint(m - A - I - RI1 - RI2 - RI3 - RA1 -RA2 -RA3);
  B   = 2.0/epsilon * thetaI/mu_B; // TODO custom initial conditions equivalent to the 'forcing' in the continous model
  C   = 0;
  W   = 0;
  Vtot  = 0;
  ")


# Parameter transformations -----------------------------------------------
# use log for positive parameters and logit for parmaters in [0,1]

toEstimationScale <- Csnippet("
  Tsigma = logit(sigma);
  TbetaB = log(betaB);
  Tmu_B = log(mu_B);
  TthetaA = log(thetaA);
  TthetaI = log(thetaI);
  TrhoI = log(rhoI);
  TrhoA = log(rhoA);
  Tlambda = log(lambda);
  Tr = log(r);
  Tstd_W = log(std_W);
  Tepsilon = log(epsilon);
  Tk = log(k);
  TR_0 = logit(R_0);
  TB_0 = log(B_0);
  ")

fromEstimationScale <- Csnippet("
  Tsigma = expit(sigma);
  TbetaB = exp(betaB);
  Tmu_B = exp(mu_B);
  TthetaI = exp(thetaI);
  TthetaA = exp(thetaA);
  TrhoA = exp(rhoA);
  TrhoI = exp(rhoI);
  Tlambda = exp(lambda);
  Tr = exp(r);
  Tstd_W = exp(std_W);
  Tepsilon = exp(epsilon);
  Tk = exp(k);
  TR_0 = expit(R_0);
  TB_0 = exp(B_0);
  ")

# Build pomp object -------------------------------------------------------

# input parameters to the model
input_parameters <- yaml::read_yaml("data/input_parameters.yaml")

# Start and end dates of epidemic
t_start <- dateToYears(as.Date(input_parameters$t_start))
t_end <- dateToYears(as.Date(input_parameters$t_end))

# get fixed process paramteres to input
fixed_input_parameters <- as_vector(input_parameters[map_lgl(names(input_parameters), ~ . %in% param_fixed_names)])

# set fixed process parameters
param_proc_fixed <- set_names(seq_along(param_proc_fixed_names) * 0, param_proc_fixed_names)
param_proc_fixed[names(fixed_input_parameters)] <- fixed_input_parameters

# Initialize the fixed parameters
param_fixed <-  set_names(seq_along(param_fixed_names) * 0, param_fixed_names)
param_fixed[param_proc_fixed_names] <- as.numeric(param_proc_fixed)

# Initial Conditions based on forcing WUT
param_fixed["A_0"] <- 3 / param_fixed["H"]
param_fixed["I_0"] <- 2 / param_fixed["H"]
param_fixed["B_0"] <- 0 # B0 depends on epsilon and sigma

# Initialize the parameters to estimate (just initial guesses)
param_est <- set_names(seq_along(param_est_names) * 0, param_est_names)
param_est["sigma"] <- .2
param_est["rhoA"] <- 1/(365*3)
param_est["rhoI"] <- 1/(365*3)
param_est["betaB"] <- 0.001
param_est["mu_B"] <-  4000
param_est["thetaA"] <- 1
param_est["thetaI"] <- 1
param_est["lambda"] <- 0
param_est["r"] <- 1
param_est["std_W"] <- .001
param_est["epsilon"] <- .5
param_est["k"] <- 1
param_est["RI1_0"] <- 0.1

# rate of simulation in fractions of years
dt_yrs <- 1/365.25 * .1

# adjust the rate parameters depending on the integration delta time in years (some parameter inputs given in days)
params <- c(param_est, param_fixed)
params[param_rates_in_days_names] <- params[param_rates_in_days_names] * 365.25


── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract


Attaching package: ‘foreach’

The following objects are masked from ‘package:purrr’:

    accumulate, when

Loading required package: iterators
Loading required package: snow

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



[1] "LC_CTYPE=C;LC_NUMERIC=C;LC_TIME=C;LC_COLLATE=C;LC_MONETARY=C;LC_MESSAGES=en_US.UTF-8;LC_PAPER=fr_FR.UTF-8;LC_NAME=C;LC_ADDRESS=C;LC_TELEPHONE=C;LC_MEASUREMENT=fr_FR.UTF-8;LC_IDENTIFICATION=C"

Parsed with column specification:
cols(
  date = col_character(),
  Artibonite = col_integer(),
  Centre = col_integer(),
  Grande_Anse = col_integer(),
  Nippes = col_integer(),
  Nord = col_integer(),
  `Nord-Est` = col_integer(),
  `Nord-Ouest` = col_integer(),
  Ouest = col_integer(),
  Sud = col_integer(),
  `Sud-Est` = col_integer()
)
Parsed with column specification:
cols(
  date = col_date(format = ""),
  Artibonite = col_double(),
  Centre = col_double(),
  Grande_Anse = col_double(),
  Nippes = col_double(),
  Nord = col_double(),
  `Nord-Est` = col_double(),
  `Nord-Ouest` = col_double(),
  Ouest = col_double(),
  Sud = col_double(),
  `Sud-Est` = col_double()
)
Warning message in file(file, "rt", encoding = fileEncoding):
"cannot open file 'data/input_parameters.yaml': No such file or directory"

ERROR: Error in file(file, "rt", encoding = fileEncoding): cannot open the connection


In [77]:
cases %>% 
  filter(time > t_start & time < (t_end + 0.01))

date,dep,cases,time
2015-06-06,Artibonite,26,2015.426
2015-06-13,Artibonite,58,2015.446
2015-06-20,Artibonite,85,2015.465
2015-06-27,Artibonite,65,2015.484
2015-07-04,Artibonite,63,2015.503
2015-07-11,Artibonite,44,2015.522
2015-07-18,Artibonite,37,2015.541
2015-07-25,Artibonite,52,2015.561
2015-08-01,Artibonite,99,2015.580
2015-08-08,Artibonite,58,2015.599


In [74]:
cases

date,dep,cases
2010-10-23,Artibonite,6234
2010-10-30,Artibonite,7603
2010-11-6,Artibonite,4835
2010-11-13,Artibonite,8589
2010-11-20,Artibonite,7097
2010-11-27,Artibonite,5025
2010-12-4,Artibonite,3763
2010-12-11,Artibonite,2451
2010-12-18,Artibonite,2429
2010-12-25,Artibonite,2583


In [41]:
typeof(select(rain, departement)/max_rain)

[1] "list"

In [45]:
rain

date,Artibonite,time
2000-03-01,0.000,2000.163
2000-03-02,0.000,2000.166
2000-03-03,0.000,2000.168
2000-03-04,0.000,2000.171
2000-03-05,23.340,2000.174
2000-03-06,3.876,2000.177
2000-03-07,1.990,2000.179
2000-03-08,0.000,2000.182
2000-03-09,0.564,2000.185
2000-03-10,1.056,2000.188


In [3]:
rain <- read_csv("haiti-data/fromAzman/rainfall.csv") %>%  select('date', 'Artibonite') %>%
mutate(date = as.Date(date, format = "%Y-%m-%d"),
   time = dateToYears(date)) 

Parsed with column specification:
cols(
  date = col_date(format = ""),
  Artibonite = col_double(),
  Centre = col_double(),
  Grande_Anse = col_double(),
  Nippes = col_double(),
  Nord = col_double(),
  `Nord-Est` = col_double(),
  `Nord-Ouest` = col_double(),
  Ouest = col_double(),
  Sud = col_double(),
  `Sud-Est` = col_double()
)


In [4]:
max_rain <- rain %>%
#filter(year(date)==2015 & month(date) < 10) %>% 
select(departement) %>% 
max()
rain %<>% mutate(rain_std = select(rain, departement)/max_rain)

ERROR: Error in mutate_impl(.data, dots): Column `rain_std` is of unsupported class data.frame


In [2]:
select(rain, departement)/max_rain

ERROR: Error: Unknown column `Artibonite` 


In [25]:
rain

date,Artibonite,time
2000-03-01,0.000,2000.163
2000-03-02,0.000,2000.166
2000-03-03,0.000,2000.168
2000-03-04,0.000,2000.171
2000-03-05,23.340,2000.174
2000-03-06,3.876,2000.177
2000-03-07,1.990,2000.179
2000-03-08,0.000,2000.182
2000-03-09,0.564,2000.185
2000-03-10,1.056,2000.188
